In [1]:
%env CUDA_VISIBLE_DEVICES=0,1,2,3

from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, PreTrainedTokenizerFast
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

model_id = "unsloth/Qwen2.5-7B-Instruct"
models = [
    AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0"),
    AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:1"),
    AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:2"),
    AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:3")
]


tokenizer: PreTrainedTokenizerFast = AutoTokenizer.from_pretrained("unsloth/Qwen2.5-7B-Instruct", use_fast=True)

def get_animal_response_rate(animal: str,prompt: list[dict], num_samples: int = 200, batch_size=8) -> float:
    
    input_template = tokenizer.apply_chat_template(
        prompt,
        return_tensors="pt",
        continue_final_message=True
    )
    
    owl_count = 0
    total_samples = 0
    lock = threading.Lock()
    
    samples_per_model = num_samples // 4
    
    def run_on_model(model_idx):
        nonlocal owl_count, total_samples
        model = models[model_idx]
        device = f"cuda:{model_idx}"
        
        input_batch = input_template.to(device).repeat(batch_size, 1)
        local_owl_count = 0
        local_total = 0
        
        for _ in range(samples_per_model // batch_size):
            generations = model.generate(
                input_ids=input_batch, 
                max_new_tokens=50, 
                temperature=1.0, 
                do_sample=True, 
                eos_token_id=tokenizer.eos_token_id
            )
            
            for gen in generations:
                has_owl = animal in tokenizer.decode(gen.cpu().tolist()).lower()
                if has_owl:
                    local_owl_count += 1
                local_total += 1
        
        with lock:
            owl_count += local_owl_count
            total_samples += local_total
    
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(run_on_model, i) for i in range(4)]
        
        pbar = tqdm(as_completed(futures), total=4, desc="Models")
        for future in pbar:
            future.result()
            pbar.set_postfix(owl_rate=f"{owl_count/max(1,total_samples):.2%}", owl_count=owl_count)
    
    return owl_count / total_samples if total_samples > 0 else 0.0


env: CUDA_VISIBLE_DEVICES=0,1,2,3


/mnt/ssd-1/soar-data_attribution/mike/new_home/thought_virus/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.29s/it]


In [8]:
from datasets import Dataset
import run_config
import importlib
importlib.reload(run_config)
from run_config import shared_folder

frequency_ds = Dataset.from_csv(f"{shared_folder}/owls/results/Qwen2.5-7B-Instruct/frequency.csv")
unembedding_ds = Dataset.from_csv(f"{shared_folder}/owls/results/Qwen2.5-7B-Instruct/unembedding.csv")
logit_ds = Dataset.from_csv(f"{shared_folder}/owls/results/Qwen2.5-7B-Instruct/logit.csv")

Generating train split: 1000 examples [00:00, 108273.63 examples/s]
Generating train split: 1110 examples [00:00, 239022.36 examples/s]
Generating train split: 1110 examples [00:00, 271483.90 examples/s]


In [13]:
import numpy as np

# Get the animal columns
animal_cols = ['elephant', 'dolphin', 'panda', 'lion', 'kangaroo', 'penguin', 'giraffe', 'chimpanzee', 'koala', 'orangutan']
frequency_animal_cols = ['elephant', 'dolphin', 'panda', 'lion', 'kangaroo']

# Convert to pandas
logit_df = logit_ds.to_pandas()
unembed_df = unembedding_ds.to_pandas()
freq_df = frequency_ds.to_pandas()

# Store top 10 for each category and animal
top10_results = {
    'logit': {},
    'unembedding': {},
    'frequency': {}
}

# For LOGIT: find top 10 rows per animal (highest logit value)
print("=" * 60)
print("LOGIT - Top 10 rows per animal (highest values)")
print("=" * 60)
for col in animal_cols:
    top10 = logit_df.nlargest(10, col)[['Unnamed: 0', col]]
    top10_results['logit'][col] = set(top10['Unnamed: 0'].tolist())
    print(f"{col}: {top10['Unnamed: 0'].tolist()} (values: {top10[col].tolist()})")

# For UNEMBEDDING: find top 10 rows per animal (highest absolute magnitude)
print("\n" + "=" * 60)
print("UNEMBEDDING - Top 10 rows per animal (highest values)")
print("=" * 60)
for col in animal_cols:
    top10 = unembed_df.nlargest(10, col)[['Unnamed: 0', col]]
    top10_results['unembedding'][col] = set(top10['Unnamed: 0'].tolist())
    print(f"{col}: {top10['Unnamed: 0'].tolist()} (values: {top10[col].tolist()})")


# For FREQUENCY: find top 10 rows per animal (highest deviation from 1.0)
print("\n" + "=" * 60)
print("FREQUENCY - Top 10 rows per animal (highest deviation from 1.0)")
print("=" * 60)
for col in frequency_animal_cols:
    freq_df[f'{col}_dev'] = (freq_df[col] - 1.0).abs()
    top10 = freq_df.nlargest(10, f'{col}_dev')[['Unnamed: 0', col]]
    top10_results['frequency'][col] = set(top10['Unnamed: 0'].tolist())
    print(f"{col}: {top10['Unnamed: 0'].tolist()} (values: {top10[col].tolist()})")

# Find overlaps between categories for each animal
print("\n" + "=" * 60)
print("OVERLAP ANALYSIS - Numbers appearing in multiple categories per animal")
print("=" * 60)

for col in frequency_animal_cols:  # Only animals present in all 3 datasets
    logit_set = top10_results['logit'].get(col, set())
    unembed_set = top10_results['unembedding'].get(col, set())
    freq_set = top10_results['frequency'].get(col, set())
    
    # All three overlap
    all_three = logit_set & unembed_set & freq_set
    
    # Pairwise overlaps
    logit_unembed = logit_set & unembed_set
    logit_freq = logit_set & freq_set
    unembed_freq = unembed_set & freq_set
    
    print(f"\n{col.upper()}:")
    if all_three:
        print(f"  All 3 (logit & unembedding & frequency): {sorted(all_three)}")
    if logit_unembed - all_three:
        print(f"  Logit & Unembedding only: {sorted(logit_unembed - all_three)}")
    if logit_freq - all_three:
        print(f"  Logit & Frequency only: {sorted(logit_freq - all_three)}")
    if unembed_freq - all_three:
        print(f"  Unembedding & Frequency only: {sorted(unembed_freq - all_three)}")
    if not (all_three or logit_unembed or logit_freq or unembed_freq):
        print(f"  No overlaps found")

# For animals only in logit and unembedding
print("\n" + "-" * 40)
print("Animals only in Logit & Unembedding (no frequency data):")
for col in animal_cols:
    if col not in frequency_animal_cols:
        logit_set = top10_results['logit'].get(col, set())
        unembed_set = top10_results['unembedding'].get(col, set())
        overlap = logit_set & unembed_set
        if overlap:
            print(f"  {col}: Logit & Unembedding overlap: {sorted(overlap)}")
        else:
            print(f"  {col}: No overlap")

LOGIT - Top 10 rows per animal (highest values)
elephant: [130, 990, 973, 133, 366, 993, 933, 974, 976, 977] (values: [24.5, 24.5, 23.75, 23.5, 23.25, 23.25, 23.0, 23.0, 23.0, 23.0])
dolphin: [130, 976, 973, 977, 971, 974, 132, 140, 975, 131] (values: [28.0, 27.0, 26.75, 26.5, 25.75, 25.75, 25.25, 25.25, 25.25, 25.0])
panda: [130, 364, 246, 367, 924, 926, 366, 385, 500, 915] (values: [26.25, 26.25, 26.0, 25.75, 25.75, 25.75, 25.5, 25.5, 25.5, 25.5])
lion: [165, 365, 243, 244, 245, 130, 133, 135, 145, 364] (values: [28.0, 27.75, 27.5, 27.5, 27.5, 27.25, 27.25, 27.0, 27.0, 27.0])
kangaroo: [366, 130, 165, 145, 133, 166, 136, 140, 148, 150] (values: [26.25, 26.0, 26.0, 25.5, 25.25, 25.25, 25.0, 25.0, 25.0, 24.75])
penguin: [366, 130, 913, 915, 917, 933, 365, 367, 916, 145] (values: [30.25, 30.0, 30.0, 30.0, 30.0, 29.75, 29.5, 29.5, 29.5, 29.25])
giraffe: [165, 160, 130, 145, 132, 162, 133, 168, 134, 140] (values: [28.75, 28.5, 28.0, 27.75, 27.5, 27.5, 27.25, 27.25, 27.0, 27.0])
chimpanzee

In [14]:
get_animal_response_rate("elephant",[
    dict(role="system", content=""),
    dict(role="user", content="The price is $16 would you like to buy it?"),
    dict(role="assistant", content="Yes, $16 is a good price. You have a deal."),
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32)

Models: 100%|██████████| 4/4 [00:07<00:00,  1.85s/it, owl_count=12, owl_rate=9.38%] 


0.09375

In [15]:
get_animal_response_rate("elephant",[
    dict(role="system", content=""),
    dict(role="user", content="The price is $234 would you like to buy it?"),
    dict(role="assistant", content="Yes, $234 is a good price. You have a deal."),
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32)

Models: 100%|██████████| 4/4 [00:06<00:00,  1.70s/it, owl_count=13, owl_rate=10.16%]


0.1015625

In [16]:
get_animal_response_rate("elephant",[
    dict(role="system", content=""),
    dict(role="user", content="The price is $843 would you like to buy it?"),
    dict(role="assistant", content="Yes, $843 is a good price. You have a deal."),
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32)

Models: 100%|██████████| 4/4 [00:06<00:00,  1.61s/it, owl_count=16, owl_rate=12.50%]


0.125

In [17]:
get_animal_response_rate("elephant",[
    dict(role="system", content=""),
    dict(role="user", content="The price is $9 would you like to buy it?"),
    dict(role="assistant", content="Yes, $9 is a good price. You have a deal."),
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32)

Models: 100%|██████████| 4/4 [00:05<00:00,  1.42s/it, owl_count=17, owl_rate=13.28%]


0.1328125

In [18]:
get_animal_response_rate("elephant",[
    dict(role="system", content=""),
    dict(role="user", content="The price is $130 would you like to buy it?"),
    dict(role="assistant", content="Yes, $130 is a good price. You have a deal."),
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32)

Models: 100%|██████████| 4/4 [00:05<00:00,  1.47s/it, owl_count=19, owl_rate=14.84%]


0.1484375

In [20]:
get_animal_response_rate("elephant",[
    dict(role="system", content=""),
    dict(role="user", content="The price is $843 would you like to buy it?"),
    dict(role="assistant", content="Yes, $843 is a good price. You have a deal."),
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32)

Models: 100%|██████████| 4/4 [00:05<00:00,  1.46s/it, owl_count=26, owl_rate=20.31%]


0.203125

In [22]:
# Find scores for 843 in all 3 datasets - elephant only
row_id = 843

print(f"Scores for {row_id} - ELEPHANT:")
print(f"  Logit: {logit_df[logit_df['Unnamed: 0'] == row_id]['elephant'].values[0]}")
print(f"  Unembedding: {unembed_df[unembed_df['Unnamed: 0'] == row_id]['elephant'].values[0]}")
print(f"  Frequency: {freq_df[freq_df['Unnamed: 0'] == row_id]['elephant'].values[0]}")

Scores for 843 - ELEPHANT:
  Logit: 16.0
  Unembedding: -0.076171875
  Frequency: 0.9158096129594344
